In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import confusion_matrix, mean_squared_error

In [2]:
PATH = "data-bluebook-for-bulldozers/"

In [3]:
!ls {PATH}

Data Dictionary.xlsx             TrainAndValid.zip
Machine_Appendix.csv             Valid.7z
Test.csv                         Valid.csv
Train.7z                         Valid.zip
Train.zip                        ValidSolution.csv
TrainAndValid.7z                 median_benchmark.csv
TrainAndValid.csv                random_forest_benchmark_test.csv


In [4]:
data_raw = pd.read_csv(f'{PATH}Train.csv',low_memory=False, 
                     parse_dates=["saledate"] )

In [5]:
data = data_raw.copy()

In [6]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [7]:
display_all(data)

SalesID  SalePrice  MachineID  ModelID  datasource  auctioneerID  \
0       1139246      66000     999089     3157         121           3.0   
1       1139248      57000     117657       77         121           3.0   
2       1139249      10000     434808     7009         121           3.0   
3       1139251      38500    1026470      332         121           3.0   
4       1139253      11000    1057373    17311         121           3.0   
...         ...        ...        ...      ...         ...           ...   
401120  6333336      10500    1840702    21439         149           1.0   
401121  6333337      11000    1830472    21439         149           1.0   
401122  6333338      11500    1887659    21439         149           1.0   
401123  6333341       9000    1903570    21435         149           2.0   
401124  6333342       7750    1926965    21435         149           2.0   

        YearMade  MachineHoursCurrentMeter UsageBand   saledate fiModelDesc  \
0           2004                      68.0       Low 2006-11-16        521D   
1           1996                    4640.0       Low 2004-03-26      950FII   
2           2001                    2838.0      High 2004-02-26         226   
3           2001                    3486.0      High 2011-05-19    PC120-6E   
4           2007                     722.0    Medium 2009-07-23        S175   
...          ...                       ...       ...        ...         ...   
401120      2005                       NaN       NaN 2011-11-02       35NX2   
401121      2005                       NaN       NaN 2011-11-02       35NX2   
401122      2005                       NaN       NaN 2011-11-02       35NX2   
401123      2005                       NaN       NaN 2011-10-25        30NX   
401124      2005                       NaN       NaN 2011-10-25        30NX   

       fiBaseModel fiSecondaryDesc fiModelSeries fiModelDescriptor  \
0              521               D           NaN               NaN   
1              950               F            II               NaN   
2              226             NaN           NaN               NaN   
3            PC120             NaN           -6E               NaN   
4             S175             NaN           NaN               NaN   
...            ...             ...           ...               ...   
401120          35              NX             2               NaN   
401121          35              NX             2               NaN   
401122          35              NX             2               NaN   
401123          30              NX           NaN               NaN   
401124          30              NX           NaN               NaN   

       ProductSize                                 fiProductClassDesc  \
0              NaN           Wheel Loader - 110.0 to 120.0 Horsepower   
1           Medium           Wheel Loader - 150.0 to 175.0 Horsepower   
2              NaN  Skid Steer Loader - 1351.0 to 1601.0 Lb Operat...   
3            Small  Hydraulic Excavator, Track - 12.0 to 14.0 Metr...   
4              NaN  Skid Steer Loader - 1601.0 to 1751.0 Lb Operat...   
...            ...                                                ...   
401120        Mini  Hydraulic Excavator, Track - 3.0 to 4.0 Metric...   
401121        Mini  Hydraulic Excavator, Track - 3.0 to 4.0 Metric...   
401122        Mini  Hydraulic Excavator, Track - 3.0 to 4.0 Metric...   
401123        Mini  Hydraulic Excavator, Track - 2.0 to 3.0 Metric...   
401124        Mini  Hydraulic Excavator, Track - 2.0 to 3.0 Metric...   

                 state ProductGroup    ProductGroupDesc Drive_System  \
0              Alabama           WL        Wheel Loader          NaN   
1       North Carolina           WL        Wheel Loader          NaN   
2             New York          SSL  Skid Steer Loaders          NaN   
3                Texas          TEX    Track Excavators          NaN   
4             New York          SSL  Skid Steer Loaders          NaN

In [21]:
display_all(data.describe(include='all').T)

count unique  \
SalesID                   401125    NaN   
SalePrice                 401125    NaN   
MachineID                 401125    NaN   
ModelID                   401125    NaN   
datasource                401125    NaN   
auctioneerID              380989    NaN   
YearMade                  401125    NaN   
MachineHoursCurrentMeter  142765    NaN   
UsageBand                  69639      3   
saledate                  401125   3919   
fiModelDesc               401125   4999   
fiBaseModel               401125   1950   
fiSecondaryDesc           263934    175   
fiModelSeries              56908    122   
fiModelDescriptor          71919    139   
ProductSize               190350      6   
fiProductClassDesc        401125     74   
state                     401125     53   
ProductGroup              401125      6   
ProductGroupDesc          401125      6   
Drive_System              104361      4   
Enclosure                 400800      6   
Forks                     192077      2   
Pad_Type                   79134      4   
Ride_Control              148606      3   
Stick                      79134      2   
Transmission              183230      8   
Turbocharged               79134      2   
Blade_Extension            25219      2   
Blade_Width                25219      6   
Enclosure_Type             25219      3   
Engine_Horsepower          25219      2   
Hydraulics                320570     12   
Pushblock                  25219      2   
Ripper                    104137      4   
Scarifier                  25230      2   
Tip_Control                25219      3   
Tire_Size                  94718     17   
Coupler                   213952      3   
Coupler_System             43458      2   
Grouser_Tracks             43362      2   
Hydraulics_Flow            43362      3   
Track_Type                 99153      2   
Undercarriage_Pad_Width    99872     19   
Stick_Length               99218     29   
Thumb                      99288      3   
Pattern_Changer            99218      3   
Grouser_Type               99153      3   
Backhoe_Mounting           78672      2   
Blade_Type                 79833     10   
Travel_Controls            79834      7   
Differential_Type          69411      4   
Steering_Controls          69369      5   
saleyear                  401125    NaN   

                                                                        top  \
SalesID                                                                 NaN   
SalePrice                                                               NaN   
MachineID                                                               NaN   
ModelID                                                                 NaN   
datasource                                                              NaN   
auctioneerID                                                            NaN   
YearMade                                                                NaN   
MachineHoursCurrentMeter                                                NaN   
UsageBand                                                            Medium   
saledate                                                2009-02-16 00:00:00   
fiModelDesc                                                            310G   
fiBaseModel                                                             580   
fiSecondaryDesc                                                           C   
fiModelSeries                                                            II   
fiModelDescriptor                                                         L   
ProductSize                                                          Medium   
fiProductClassDesc        Backhoe Loader - 14.0 to 15.0 Ft Standard Digg...   
state                                                               Florida   
ProductGroup                                                            TEX   
ProductGroupDesc                                           Track Excavators   
Dr

In [8]:
data.SalePrice = np.log(data.SalePrice)

# Data Preprocessing

This dataset contains a mix of continuous and categorical variables.

The following method extracts particular date fields from a complete datetime for the purpose of constructing categoricals. You should always consider this feature extraction step when working with date-time. Without expanding your date-time into these additional fields, you can't capture any trend/cyclical behavior as a function of time at any of these granularities.

In [9]:
data['saleyear'] = data['saledate'].dt.year

In [10]:
data.saleyear.head()

0    2006
1    2004
2    2004
3    2011
4    2009
Name: saleyear, dtype: int64

In [11]:
type(data['saleyear'][0])

numpy.int64

We have lots of missing values, which we can't pass directly to a random forest. So first we will find how many missing values we have followed by dealing with it.

In [13]:
display_all(data.isnull().sum().sort_index()/len(data))

Backhoe_Mounting            0.803872
Blade_Extension             0.937129
Blade_Type                  0.800977
Blade_Width                 0.937129
Coupler                     0.466620
Coupler_System              0.891660
Differential_Type           0.826959
Drive_System                0.739829
Enclosure                   0.000810
Enclosure_Type              0.937129
Engine_Horsepower           0.937129
Forks                       0.521154
Grouser_Tracks              0.891899
Grouser_Type                0.752813
Hydraulics                  0.200823
Hydraulics_Flow             0.891899
MachineHoursCurrentMeter    0.644089
MachineID                   0.000000
ModelID                     0.000000
Pad_Type                    0.802720
Pattern_Changer             0.752651
ProductGroup                0.000000
ProductGroupDesc            0.000000
ProductSize                 0.525460
Pushblock                   0.937129
Ride_Control                0.629527
Ripper                      0.740388
S

In [ ]:
data.fillna(method='')

The categorical variables are currently stored as strings, which is inefficient, and doesn't provide the numeric coding required for a random forest. Therefore we call train_cats to convert strings to pandas categories.

In [14]:
data.UsageBand.head()

0       Low
1       Low
2      High
3      High
4    Medium
Name: UsageBand, dtype: object

In [15]:
type(data.UsageBand[0])

str

In [16]:
data.UsageBand = pd.Categorical(data.UsageBand, 
                                categories=['High', 'Medium', 'Low'], ordered=True)

In [17]:
data.UsageBand.head()

0       Low
1       Low
2      High
3      High
4    Medium
Name: UsageBand, dtype: category
Categories (3, object): [High < Medium < Low]

In [ ]:
categorical_label = data.UsageBand.to_list()
categorical_label

In [18]:
le = LabelEncoder()

In [ ]:
forr = RandomForestRegressor(n_jobs=-1)

In [ ]:
forr.fit(data.drop('SalePrice',axis=1), data.SalePrice)

In [ ]:
for i in data:
    if data[i] == '<':
        print(i)
    else:
        print('none')